# ETL da Camada Silver para Camada Gold

No caso desse ETL, a fonte será extraida do esquema da camada silver, a limpeza será escassa ou até nula e o carregamento será armazenado num novo esquema do banco.

### Frameworks utilizados

In [56]:
import pandas as pd
import psycopg2
import time

### Conexão com a Silver

Conectando com o banco

In [57]:
def get_db_connection():
    while True:
        try:
            conexao = psycopg2.connect(
                host="localhost",
                port=5432,
                database="steam_bi",
                user="steam_bi_user",
                password="steam_bi_user"
            )
            return conexao
        except psycopg2.OperationalError:
            print("O banco não está pronto, aguardando 3 segundos...")
            time.sleep(3)


conexao = get_db_connection()

print("Conexão efetuada com sucesso...")

Conexão efetuada com sucesso...


Extraindo dados do banco

In [63]:
df = pd.read_sql_query("SELECT * FROM public.tb_games_silver", conexao)

print("Dados carregados do banco de dados:")
print(df.info())
print(f"Total de registros: {len(df)}")

print("\n Extract concluído!")

/tmp/ipykernel_61491/906146711.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM public.tb_games_silver", conexao)


Dados carregados do banco de dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122610 entries, 0 to 122609
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id_jogo             122610 non-null  int64  
 1   nome_jogo           122610 non-null  object 
 2   idade_minima        122610 non-null  int64  
 3   preco               122610 non-null  float64
 4   nota_metacritic     122610 non-null  int64  
 5   nota_usuario        122610 non-null  int64  
 6   desenvolvedores     122610 non-null  object 
 7   publicadoras        122610 non-null  object 
 8   categorias          122610 non-null  object 
 9   generos             122610 non-null  object 
 10  tags                122610 non-null  object 
 11  ano_lancamento      122610 non-null  int64  
 12  tier_preco          122610 non-null  object 
 13  tem_ptbr_interface  122610 non-null  bool   
 14  tem_ptbr_audio      122610 non-null  bool   
dty

### Definição de Dados 

Criando schema dw e tabelas

In [59]:
cursor = conexao.cursor()

# cria um schema para a silver
cursor.execute("""
DROP SCHEMA IF EXISTS DW CASCADE;
CREATE SCHEMA DW;

COMMENT ON SCHEMA DW IS 'Data Warehouse (DW), banco de dados atuante na camada gold - Dados estruturados, agregados e otimizados para análise histórica';
""")

# cria Dim_pokmn (POKEMON)
cursor.execute("""
CREATE TABLE DW.DIM_OFR(
    SRK_OFR BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    VLR_PRC NUMERIC(8, 2) NOT NULL,
    TIR_PRC VARCHAR(50) NOT NULL
);


COMMENT ON TABLE DW.DIM_OFR IS 'Dimensão Oferta - Informações econômicas sobre os jogos';
COMMENT ON COLUMN DW.DIM_OFR.SRK_OFR IS 'Surrogate Key - Chave primária da oferta';
COMMENT ON COLUMN DW.DIM_OFR.VLR_PRC IS 'Preço do jogo - Valor';
COMMENT ON COLUMN DW.DIM_OFR.TIR_PRC IS 'Tier do Preço - Gratuito, Budget (<$10), Indie / Padrão ($10-$29), Double-A / Premium ($30-$58), AAA / Lançamento (+$59)';
""")

# cria Dim_batlh (BATALHA)
cursor.execute("""
CREATE TABLE DW.DIM_DEV(
    SRK_DEV BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    NME_DEV VARCHAR(1000) NOT NULL
);

COMMENT ON TABLE DW.DIM_DEV IS 'Dimensão Desenvolvedor - Informações sobre desenvolvedores dos jogos';
COMMENT ON COLUMN DW.DIM_DEV.SRK_DEV IS 'Surrogate Key - Chave primária da lista de desenvolvedores';
COMMENT ON COLUMN DW.DIM_DEV.NME_DEV IS 'Lista de nomes dos desenvolvedores de cada jogo';
""")

# cria Dim_efetContr (EFETIVIDADE CONTRA)
cursor.execute("""
CREATE TABLE DW.DIM_PBS(
    SRK_PBS BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    NME_PBS VARCHAR(300) NOT NULL
);

COMMENT ON TABLE DW.DIM_PBS IS 'Dimensão Publicador - Informações sobre publicadoras dos jogos';
COMMENT ON COLUMN DW.DIM_PBS.SRK_PBS IS 'Surrogate Key - Chave primária da lista de publicadoras';
COMMENT ON COLUMN DW.DIM_PBS.NME_PBS IS 'Lista de nomes das publicadoras de cada jogo';
""")

cursor.execute("""
CREATE TABLE DW.DIM_CAT(
    SRK_CAT BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    NME_CAT VARCHAR(1000) NOT NULL
);

COMMENT ON TABLE DW.DIM_CAT IS 'Dimensão Categoria - Informações sobre categorias dos jogos';
COMMENT ON COLUMN DW.DIM_CAT.SRK_CAT IS 'Surrogate Key - Chave primária da lista de categorias de cada jogo';
COMMENT ON COLUMN DW.DIM_CAT.NME_CAT IS 'Lista de nomes das categorias de cada jogo';
""")

cursor.execute("""
CREATE TABLE DW.DIM_GEN(
    SRK_GEN BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    NME_GEN VARCHAR(500) NOT NULL
);

COMMENT ON TABLE DW.DIM_GEN IS 'Dimensão Gênero - Informações sobre gêneros dos jogos';
COMMENT ON COLUMN DW.DIM_GEN.SRK_GEN IS 'Surrogate Key - Chave primária da lista de gêneros de cada jogo';
COMMENT ON COLUMN DW.DIM_GEN.NME_GEN IS 'Lista de nomes dos gêneros de cada jogo';
""")

cursor.execute("""
CREATE TABLE DW.DIM_TAG(
    SRK_TAG BIGINT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    NME_TAG VARCHAR(500) NOT NULL
);

COMMENT ON TABLE DW.DIM_TAG IS 'Dimensão Tag - Informações sobre tags dos jogos';
COMMENT ON COLUMN DW.DIM_TAG.SRK_TAG IS 'Surrogate Key - Chave primária da lista de tags de cada jogo';
COMMENT ON COLUMN DW.DIM_TAG.NME_TAG IS 'Lista de nomes de tags de cada jogo';
""")

# cria Fat_pokdx (POKEDEX)
cursor.execute("""
CREATE TABLE DW.FAT_JGO(
    SRK_JGO BIGINT PRIMARY KEY,
    NME_JGO VARCHAR(500) NOT NULL,
    VLR_IDD SMALLINT NOT NULL,
    TEM_PTB_ITF BOOLEAN NOT NULL,
    TEM_PTB_AUD BOOLEAN NOT NULL,
    NTA_USR SMALLINT NOT NULL,
    NTA_MTC SMALLINT NOT NULL,
    ANO_LNC SMALLINT NOT NULL,
    SRK_OFR BIGINT NOT NULL REFERENCES DW.DIM_OFR(SRK_OFR),
    SRK_DEV BIGINT NOT NULL REFERENCES DW.DIM_DEV(SRK_DEV),
    SRK_PBS BIGINT NOT NULL REFERENCES DW.DIM_PBS(SRK_PBS),
    SRK_CAT BIGINT NOT NULL REFERENCES DW.DIM_CAT(SRK_CAT),
    SRK_GEN BIGINT NOT NULL REFERENCES DW.DIM_GEN(SRK_GEN),
    SRK_TAG BIGINT NOT NULL REFERENCES DW.DIM_TAG(SRK_TAG)
);

COMMENT ON TABLE DW.FAT_JGO IS 'Fato Jogo - Informações de jogos';
COMMENT ON COLUMN DW.FAT_JGO.SRK_JGO IS 'Surrogate Key -  Chave primária de cada jogo, referente ao ID do jogo na Steam';
COMMENT ON COLUMN DW.FAT_JGO.NME_JGO IS 'Nome do jogo';
COMMENT ON COLUMN DW.FAT_JGO.VLR_IDD IS 'Idade indicada para jogar';
COMMENT ON COLUMN DW.FAT_JGO.TEM_PTB_ITF IS 'Tem Interface em Português?';
COMMENT ON COLUMN DW.FAT_JGO.TEM_PTB_AUD IS 'Tem Áudio em Português?';
COMMENT ON COLUMN DW.FAT_JGO.NTA_USR IS 'Nota do jogo pelos usuários da Steam';
COMMENT ON COLUMN DW.FAT_JGO.NTA_MTC IS 'Nota do jogo pelo Metacritic';
COMMENT ON COLUMN DW.FAT_JGO.ANO_LNC IS 'Ano de Lançamento';
COMMENT ON COLUMN DW.FAT_JGO.SRK_OFR IS 'Chave estrangeira sorrogate para DIM_OFR - Referente a Dimensão Oferta';
COMMENT ON COLUMN DW.FAT_JGO.SRK_DEV IS 'Chave estrangeira sorrogate para DIM_DEV - Referente a Dimensão Desenvolvedor';
COMMENT ON COLUMN DW.FAT_JGO.SRK_PBS IS 'Chave estrangeira sorrogate para DIM_PBS - Referente a Dimensão Publicadora';
COMMENT ON COLUMN DW.FAT_JGO.SRK_CAT IS 'Chave estrangeira sorrogate para DIM_CAT - Referente a Dimensão Categoria';
COMMENT ON COLUMN DW.FAT_JGO.SRK_GEN IS 'Chave estrangeira sorrogate para DIM_GEN - Referente a Dimensão Gênero';
COMMENT ON COLUMN DW.FAT_JGO.SRK_TAG IS 'Chave estrangeira sorrogate para DIM_TAG - Referente a Dimensão Tag';

""")

conexao.commit()

### Populando o Banco

In [60]:
# Usa a mesma conexao e cursor da célula anterior

# 1) Inserção na DIM_OFR
for _, row in df.iterrows():
    try:
        cursor.execute("""
            INSERT INTO DW.DIM_OFR (
                VLR_PRC, TIR_PRC
            ) VALUES (%s, %s)
            RETURNING SRK_OFR
        """, (
            row["preco"], row["tier_preco"]
        ))
        srk_ofr = cursor.fetchone()[0]
    
    except Exception as e:
        print("FAILED DIM_DEV:", row["desenvolvedores"])
        print(e.pgerror)
        conexao.rollback()
        raise
    # 2) Inserção na DIM_DEV
    cursor.execute("""
        INSERT INTO DW.DIM_DEV (
            NME_DEV
        ) VALUES (%s)
        RETURNING SRK_DEV
    """, (
        row["desenvolvedores"],
    ))
    srk_dev = cursor.fetchone()[0]

    # 3) Inserção na DIM_PBS
    cursor.execute("""
        INSERT INTO DW.DIM_PBS (
            NME_PBS
        ) VALUES (%s)
        RETURNING SRK_PBS
    """, (
        row["publicadoras"],
    ))
    srk_pbs = cursor.fetchone()[0]

    # 4) Inserção na DIM_CAT
    cursor.execute("""
        INSERT INTO DW.DIM_CAT (
            NME_CAT
        ) VALUES (%s)
        RETURNING SRK_CAT
    """, (
        row["categorias"],
    ))
    srk_cat = cursor.fetchone()[0]

    # 5) Inserção na DIM_GEN
    cursor.execute("""
        INSERT INTO DW.DIM_GEN (
            NME_GEN
        ) VALUES (%s)
        RETURNING SRK_GEN
    """, (
        row["generos"],
    ))
    srk_gen = cursor.fetchone()[0]

    # 6) Inserção na DIM_TAG
    cursor.execute("""
        INSERT INTO DW.DIM_TAG (
            NME_TAG
        ) VALUES (%s)
        RETURNING SRK_TAG
    """, (
        row["tags"],
    ))
    srk_tag = cursor.fetchone()[0]

    # 7) Inserção na Tabela Fato
    cursor.execute("""
        INSERT INTO DW.FAT_JGO (
            SRK_JGO, NME_JGO, VLR_IDD, TEM_PTB_ITF, TEM_PTB_AUD, NTA_USR, NTA_MTC, ANO_LNC, SRK_OFR, SRK_DEV, SRK_PBS, SRK_CAT, SRK_GEN, SRK_TAG
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row["id_jogo"], row["nome_jogo"], row["idade_minima"], row["tem_ptbr_interface"], row["tem_ptbr_audio"], row["nota_usuario"], row["nota_metacritic"], row["ano_lancamento"], srk_ofr, srk_dev, srk_pbs, srk_cat, srk_gen, srk_tag))


# Commit final
conexao.commit()



Fechando a conexão com o banco!

In [9]:
conexao.close()

print("\n Load concluído!")


 Load concluído!
